## Scraping BT for Novo Nordisk

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import time
import re
import requests
import pandas as pd

In [138]:
#Get the response
#response=requests.get('https://www.bt.dk', headers={'name': 'Lea Lund Jeppesen', 'e-mail':'snk956@alumni.ku.dk'})

# Set up the driver and navigate to the URL
url = 'https://www.bt.dk'
driver = webdriver.Chrome()  # Ensure you have the ChromeDriver set up correctly
driver.get(url,)

# Find and click on the "Allow Cookies" button
allow_cookies_button = driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyButtonDecline")
allow_cookies_button.click()

#Find and click the menu button
menu_data_id = "menuToggler"
menu_element = driver.find_element(By.CSS_SELECTOR, f"[data-id='{menu_data_id}']")
menu_element.click()

# Locate the search input element by its ID
search_input_id = "header-search"
search_input = driver.find_element(By.ID, search_input_id)

# Perform a search action in the search input
search_input.send_keys("Novo Nordisk")  # Replace with your search query
search_input.submit()






[]


In [148]:
# Get the page source
page_source = driver.page_source

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(page_source, "html.parser")

total_articles=soup.find_all("h1", {"class":"page-title"})[0].text.split(" ")[1]
total_articles


'1480'

In [154]:
steps=[]

for i in range(0,int(1480/100)+1): 
    steps.append(i*100)
    
steps.append(int(total_articles))


[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1480]


In [234]:
links=[]

steps=[0]
for i in steps: 
    response=requests.get(f'https://www.bt.dk/search?query=Novo%20Nordisk&limit=100&offset={i}')
    
    # Use Beautiful Soup to parse the HTML
    soup = BeautifulSoup(response.content, "lxml")
    links.append(str(soup.find_all(class_='teaser teaser--flat')))
    
    time.sleep(0.1)

print(links[0].split("href"))   


#for child in links[0][0]:
 #   print(child)
  #  print('next')
    
#links[0][0].find('a')
#links[0][0].contents[1]

['[<article class="teaser teaser--flat">\n<a class="teaser__link" data-k5a-url="https://www.bt.dk/content/item/1708295" data-lpurl="https://www.bt.dk/content/item/1708295" ', '="https://www.bt.dk/forbrug/afgoerende-for-danmark-novo-nordisk-krydser-utrolig-graense-efter-nye-jubeltal">\n<figure class="teaser__image">\n<img alt="Afgørende for Danmark: Novo Nordisk krydser utrolig grænse efter nye jubeltal" class="img-responsive" src="https://bt.bmcdn.dk/media/cache/resolve/image_300x200/image/204/2047127/24496245-novo-nordisk-fejrer-100-rs-fdselsdag-med-et-kmp.jpg"/>\n</figure>\n<div class="teaser-body">\n<h2 class="teaser-body__header">\n                Afgørende for Danmark: Novo Nordisk krydser utrolig grænse efter nye jubeltal\n            </h2>\n</div>\n</a>\n</article>, <article class="teaser teaser--flat">\n<a class="teaser__link" data-k5a-url="https://www.bt.dk/content/item/1708311" data-lpurl="https://www.bt.dk/content/item/1708311" ', '="https://www.bt.dk/erhverv/novo-nordisk-ka

In [230]:
links=[]

steps=[0]
for i in steps: 
    response=requests.get(f'https://www.bt.dk/search?query=Novo%20Nordisk&limit=100&offset={i}')
    
    # Use Beautiful Soup to parse the HTML
    soup = BeautifulSoup(response.content, "lxml")
    links.append(soup.find_all('href='))
    time.sleep(0.1)

print(links)

[[]]


In [ ]:
    
loop=0
while True:
    try:
        time.sleep(1)
        loop+=1 
        load_more_button = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div[3]/div[2]/button')
        load_more_button.click()
        if loop == 10: 
            break
    except:
        print('error')
        break

In [50]:
# Get the page source
page_source = driver.page_source

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(page_source, "html.parser")

# Find all articles
articles = soup.find_all("article", class_="teaser teaser--flat")

# Define a list to store data
article_data = []

# Loop through articles
for article in articles:
    # Extract the link to the article
    article_link = article.find("a", class_="teaser__link")["href"]
    
    # Click the article link to view it
    driver.get(article_link)
    time.sleep(2)  # Wait for article content to load
    
    # Extract title and date from the opened article
    article_soup = BeautifulSoup(driver.page_source, "html.parser")
    title = article_soup.find("h1", class_="article-title").text.strip()
    date = article_soup.find("div", class_="article-date").text.strip()
    
    # Store the title and date in the list
    article_data.append({title, date})
    
    # Go back to the search results
    driver.back()
    time.sleep(2)  # Wait for demonstration purposes


In [127]:
# Create a pandas DataFrame
df = pd.DataFrame(article_data, columns=["Title", "Date"])
df 

    

        



,Title,Date
0,Novo Nordisk kæmper med at følge den store eft...,2 timer siden\n | Opdateret...
1,Novo Nordisk-aktien er gået amok: 10.000 krone...,2 timer siden
2,Novos gigantoverskud kommer hele Danmark til gode,2 timer siden\n | Opdateret...
3,Novo Nordisk hæver milliardoverskud trods udbu...,5 timer siden\n | Opdateret...
4,Salg af Novo-slankemiddel boomer: Mere end sek...,5 timer siden
5,Bagsiden af Novo Nordisks succes: Millionforli...,9. aug. 2023 - 17:26
6,Regioner forventer milliardregning for Novo-me...,9. aug. 2023 - 7:01\n \n ...
7,Novos markedsværdi stiger med over 400 milliar...,8. aug. 2023 - 17:37\n \n ...
8,Testresultater sender Novo-aktien i rekordniveau,8. aug. 2023 - 13:23\n \n ...
9,Novo sagsøgt: Kvinde oplever mavelammelse i ja...,3. aug. 2023 - 10:05\n \n ...
